In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('full_dataset.csv')

In [ ]:
df.drop(columns='player_name', inplace=True)
df['minutes'] = df['minutes'].str.replace(',', '')
df['minutes'] = pd.to_numeric(df['minutes'])
df.fillna(df.mean(), inplace=True)


<ipython-input-104-28b2f6b74f5e>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [ ]:
Q1 = df["fee_cleaned"].quantile(0.25)
Q3 = df["fee_cleaned"].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df["fee_cleaned"] < (Q1 - 1.5 * IQR)) | (df["fee_cleaned"] > (Q3 + 1.5 * IQR)))]

df.dropna(subset=["fee_cleaned"], inplace=True)

In [ ]:
le = LabelEncoder()
df['position'] = le.fit_transform(df['position'])
df['club_involved_name'] = le.fit_transform(df['club_involved_name'])
df['club_name'] = le.fit_transform(df['club_name'])
df['transfer_movement'] = le.fit_transform(df['transfer_movement'])
df['season'] = le.fit_transform(df['season'])
df['nationality'] = le.fit_transform(df['nationality'])

In [ ]:
features = df.drop(columns='fee_cleaned')
features.head()


,age,position,club_name,club_involved_name,transfer_movement,matches_played,goals,assists,yellow_card,red_card,...,touches,carries,goals_against,goals_against90,Saves,Saves%,PSxG,plus_minus90,season,nationality
0,31,8,145,4,0,37,2.0,0.0,6.0,1.0,...,2759.0,1588.0,0.0,0.0,0.0,0.0,0.0,0.75,0,23
1,28,2,12,30,0,33,1.0,4.0,4.0,0.0,...,2532.0,1526.0,0.0,0.0,0.0,0.0,0.0,0.99,0,23
2,27,2,77,55,1,25,2.0,3.0,0.0,0.0,...,293.0,182.0,0.0,0.0,0.0,0.0,0.0,0.17,1,23
3,25,0,145,43,0,39,0.0,1.0,2.0,0.0,...,2377.0,1534.0,0.0,0.0,0.0,0.0,0.0,0.48,1,23
4,29,12,86,94,0,34,5.0,1.0,5.0,0.0,...,175.0,85.0,0.0,0.0,0.0,0.0,0.0,0.12,1,23


In [ ]:
target = df['fee_cleaned']
target.head()

0     2.5
1     0.8
2     2.0
3    13.5
4     2.7
Name: fee_cleaned, dtype: float64

In [ ]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

In [ ]:
mse_scores = []
r2_scores = []

In [ ]:
for train_index, test_index in kf.split(features):
    features_train, features_test = features.iloc[train_index], features.iloc[test_index]
    target_train, target_test = target.iloc[train_index], target.iloc[test_index]
    #train the model for each fold
    svm_model = SVR(kernel='rbf', C=87)
    svm_model.fit(features_train, target_train)
    #prediction for each fold
    target_pred = svm_model.predict(features_test)
    #calculate error
    mse = mean_squared_error(target_test, target_pred)
    r2 = r2_score(target_test, target_pred)
    #add to lists
    mse_scores.append(mse)
    r2_scores.append(r2)


In [ ]:
mean_mse = np.mean(mse_scores)
mean_r2 = np.mean(r2_scores)
std_mse = np.std(mse_scores)
std_r2 = np.std(r2_scores)

print(f"Mean MSE: {mean_mse:.2f} (±{std_mse:.2f})")
print(f"Mean R-squared: {mean_r2:.2f} (±{std_r2:.2f})")

Mean MSE: 127.28 (±19.94)
Mean R-squared: 0.06 (±0.08)
